# ONE basic commands

There are four main ONE methods that can be used to search for and download data. Here we provide an introduction to these commands using some simple example implementations. Before proceeding make sure that you have installed the [IBL python environment](../../02_installation.md) and set up your [ONE credentials](../../one_docs/one_credentials.md)

## 1. Importing ONE
To use ONE, we must first import the module and instantiate the ONE class. This can be done using the following commands


In [1]:
from oneibl.one import ONE
one = ONE()

Connected to https://alyx.internationalbrainlab.org as mayo


<div class="alert alert-info">

Note

If you are trying to connect through your personal IP address you may recieve the following error

```
ConnectionError: Can't connect to https://alyx.internationalbrainlab.org.
IP addresses are filtered on IBL database servers.
Are you connecting from an IBL participating institution ?
```
If so, make sure you VPN into an IBL institution network

</div>


## 2. Searching for data

Each experimental session is identified by a unique string known as an experiment ID (eID). To search for experiments, we can use the `one.search` command. This will return a list of eIDs that match the search criterion. 

The possible searchable fields can be listed using the method

In [2]:
one.search_terms()

['dataset_types',
 'date_range',
 'lab',
 'location',
 'number',
 'performance_gte',
 'performance_lte',
 'subject',
 'task_protocol',
 'users']

Let's search for the experiments performed with the subject KS022 on the 10 December 2019

In [2]:
eid = one.search(subject='KS022', date_range=['2019-12-10', '2019-12-10'])
print(f"experiment ID = {eid}")

experiment ID = ['a3df91c8-52a6-4afa-957b-3479a7d0897c']


To get more information about the returned sessions, we can add in a `details` flag

In [25]:
eid, ses_info = one.search(subject='KS022', date_range=['2019-12-10', '2019-12-10'], details=True)
print(ses_info)

[{'subject': 'KS022', 'start_time': '2019-12-10T14:40:57.879000', 'number': 1, 'lab': 'cortexlab', 'project': 'carandiniharris_midbrain_ibl', 'url': 'https://alyx.internationalbrainlab.org/sessions/a3df91c8-52a6-4afa-957b-3479a7d0897c', 'task_protocol': '_iblrig_tasks_ephysChoiceWorld6.1.3', 'local_path': '[]\\cortexlab\\Subjects\\KS022\\2019-12-10\\001'}]


## 3. Listing available data

Once the eID of the session of interest is known, we can list the available datasets using the `one.list` command

In [26]:
one.list(eid)

[['_ibl_log.info',
  '_iblqc_ephysSpectralDensity.freqs',
  '_iblqc_ephysSpectralDensity.power',
  '_iblqc_ephysTimeRms.rms',
  '_iblqc_ephysTimeRms.timestamps',
  '_iblrig_Camera.raw',
  '_iblrig_Camera.timestamps',
  '_iblrig_ambientSensorData.raw',
  '_iblrig_codeFiles.raw',
  '_iblrig_encoderEvents.raw',
  '_iblrig_encoderPositions.raw',
  '_iblrig_encoderTrialInfo.raw',
  '_iblrig_taskData.raw',
  '_iblrig_taskSettings.raw',
  '_phy_spikes_subset.channels',
  '_phy_spikes_subset.spikes',
  '_phy_spikes_subset.waveforms',
  '_spikeglx_sync.channels',
  '_spikeglx_sync.polarities',
  '_spikeglx_sync.times',
  'camera.times',
  'channels.localCoordinates',
  'channels.rawInd',
  'clusters.amps',
  'clusters.channels',
  'clusters.depths',
  'clusters.metrics',
  'clusters.peakToTrough',
  'clusters.uuids',
  'clusters.waveforms',
  'clusters.waveformsChannels',
  'ephysData.raw.ap',
  'ephysData.raw.ch',
  'ephysData.raw.lf',
  'ephysData.raw.meta',
  'ephysData.raw.sync',
  'ephysDa

We can set the `details` flag to True to return more information about each dataset

In [12]:
data_details = one.list(eid, details=True)
data_details[0][0]

SessionDataInfo(dataset_type='ephysData.raw.ap', dataset_id='8e162391-cf35-4329-b28d-e4be55d57d56', local_path=None, eid='a3df91c8-52a6-4afa-957b-3479a7d0897c', url='http://ibl.flatironinstitute.org/cortexlab/Subjects/KS022/2019-12-10/001/raw_ephys_data/probe01/_spikeglx_ephysData_g0_t0.imec1.ap.8e162391-cf35-4329-b28d-e4be55d57d56.cbin', data=None, hash=[], file_size=[])

We can also use the `one.list` command to help us navigate the database by finding the list of possible keywords that we can use to refine our search for sessions. For example, to list the labs available, we can type

In [29]:
one.list(None, 'labs')

['hoferlab',
 'mainenlab',
 'angelakilab',
 'wittenlab',
 'danlab',
 'cortexlab',
 'churchlandlab',
 'mrsicflogellab',
 'zadorlab',
 'steinmetzlab']

## 4. Loading data

Once we have information about the session eID and the list of available dataset types we can use the `one.load` function to download the data from FlatIron. Let's download the `clusters.amps` and `clusters.depths` datasets


In [3]:
dataset_types = ['clusters.amps', 'clusters.depths']
amps0, depths0, amps1, depths1 = one.load(eid, dataset_types=dataset_types)


ValueError: not enough values to unpack (expected 4, got 1)

Note. Loading `clusters.amps` and `clusters.depths` returned four variables. This is because there is data for two different probes for this subject. 

To find out more details about what is being downloaded we can return a data class that contains information about each of the downloaded datasets. This can be added by setting the `dclass_output` argument to True. 

In [4]:
data = one.load(eid, dataset_types=dataset_types, dclass_output=True)

We have access to the following information

In [21]:
data.__dict__.keys()

dict_keys(['dataset_type', 'dataset_id', 'local_path', 'eid', 'url', 'data', 'hash', 'file_size'])

For example if we look at the local_path we will see that the data has come from two different probes, `probe00` and `probe01`

In [23]:
data.local_path

[WindowsPath('C:/Users/Mayo/Downloads/FlatIron/cortexlab/Subjects/KS022/2019-12-10/001/alf/probe01/clusters.depths.npy'),
 WindowsPath('C:/Users/Mayo/Downloads/FlatIron/cortexlab/Subjects/KS022/2019-12-10/001/alf/probe00/clusters.amps.npy'),
 WindowsPath('C:/Users/Mayo/Downloads/FlatIron/cortexlab/Subjects/KS022/2019-12-10/001/alf/probe01/clusters.amps.npy'),
 WindowsPath('C:/Users/Mayo/Downloads/FlatIron/cortexlab/Subjects/KS022/2019-12-10/001/alf/probe00/clusters.depths.npy')]

<div class="alert alert-info">

Tip

To find out more information about the ONE functions and the arguments that can be parsed we can use the help command. For example we can type

</div>

In [25]:
help(one.load)

Help on method load in module oneibl.one:

load(eid, dataset_types=None, dclass_output=False, dry_run=False, cache_dir=None, download_only=False, clobber=False, offline=False, keep_uuid=False) method of oneibl.one.ONE instance
    From a Session ID and dataset types, queries Alyx database, downloads the data
    from Globus, and loads into numpy array.
    
    :param eid: Experiment ID, for IBL this is the UUID of the Session as per Alyx
     database. Could be a full Alyx URL:
     'http://localhost:8000/sessions/698361f6-b7d0-447d-a25d-42afdef7a0da' or only the UUID:
     '698361f6-b7d0-447d-a25d-42afdef7a0da'. Can also be a list of the above for multiple eids.
    :type eid: str
    :param dataset_types: [None]: Alyx dataset types to be returned.
    :type dataset_types: list
    :param dclass_output: [False]: forces the output as dataclass to provide context.
    :type dclass_output: bool
     If None or an empty dataset_type is specified, the output will be a dictionary by defaul